In [225]:
import requests
import os
import json
from typing import Dict
# URL to fetch the JSON data
url = "https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_3_0_0/export/json?element=all"

# File path to store the JSON data
framework_v3_file_path = "../client/public/data/sp_800_171_3_0_0/framework.json"

# Fetch the JSON data
response = requests.get(url)
framework_data = response.json()

In [226]:
withdrawn: Dict[str, str] = {}
for element in framework_data["response"]["elements"]["elements"]:
    if element['element_type'] == "withdraw_reason":
        element_id = element["element_identifier"][-8:]
        withdrawn[element_id] = element["text"]

In [227]:
# File path to store the JSON data
framework_v2_file_path = "../client/public/data/sp_800_171_2_0_0/framework.json"

# Write the JSON data to the file
with open(framework_v2_file_path, 'r') as file:
   v2 = json.load(file)

In [228]:
_v2_requirements = [ 
    element for element 
    in v2["response"]["elements"]["elements"] 
    if element["element_type"] == "requirement"
]
v2_requirements = {
   requirement['element_identifier']:
   requirement for requirement in _v2_requirements
}

_v3_requirements = [ 
    element for element
    in framework_data["response"]["elements"]["elements"] 
    if element["element_type"] == "requirement"
]
v3_requirements = {
    requirement['element_identifier']:
    requirement for requirement in _v3_requirements
}

In [229]:
import re
from typing import Dict

pattern = re.compile(r'\d{2}\.\d{2}\.\d{2}')

value_mapping: Dict[str, Dict] = {}

for identifier, text in withdrawn.items():
    matches = pattern.findall(text)
    
    if identifier not in value_mapping:
        value_mapping[identifier] = {
            "withdrawn_into": list(matches),
            "withdrawn_from": list(),
            "value": 0,
            "partial_value": 0,
            "revision": [],
            "aggregate_value_withdrawn_from": 0,
            "aggregate_partial_value_withdrawn_from": 0
        }
    else:
        value_mapping[identifier]["withdrawn_into"] = value_mapping[identifier]["withdrawn_into"] + list(matches)

    for match in matches:
        if match not in value_mapping:
            value_mapping[match] = {
                "withdrawn_into": list(),
                "withdrawn_from": [identifier],
                "value": 0,
                "partial_value": 0,
                "revision": [],
                "aggregate_value_withdrawn_from": 0,
                "aggregate_partial_value_withdrawn_from": 0
            }
        else:
            value_mapping[match]["withdrawn_from"].append(identifier)

for element in _v3_requirements:
    if element["element_identifier"] not in value_mapping:
        value_mapping[element["element_identifier"]] = {
                "withdrawn_into": list(),
                "withdrawn_from": list(),
                "value": 0,
                "partial_value": 0,
                "revision": [],
                "aggregate_value_withdrawn_from": 0,
                "aggregate_partial_value_withdrawn_from": 0
        }

In [230]:
%%bash
# Scoring https://www.acq.osd.mil/asda/dpc/cp/cyber/docs/safeguarding/NIST-SP-800-171-Assessment-Methodology-Version-1.2.1-6.24.2020.pdf
mlr --icsv --ojson --jlistwrap cat ../data/scores.csv > ../data/scores.json

In [231]:
def _get_revisions(identifier: str):
    if identifier in withdrawn:
        return [2]

    if identifier in v2_requirements and identifier in v3_requirements:
        return [2,3]
    
    return [3]

for [identifier, item] in value_mapping.items():
    item['revision'] = _get_revisions(identifier)

In [232]:
# Write the JSON data to the file
with open("../data/scores.json", 'r') as file:
   scores = json.load(file)

for score in scores:
   identifier = score["identifier"]
   value = score["value"]
   partial_value = score["partial_value"]

   item = value_mapping[identifier]
   into_count = len(item["withdrawn_into"])

   if into_count:
      for into_id in item["withdrawn_into"]:
         value_mapping[into_id]["aggregate_value_withdrawn_from"] += (value / into_count)
         value_mapping[into_id]["aggregate_partial_value_withdrawn_from"] += (partial_value / into_count)

   item["value"] = value
   item["partial_value"] = partial_value

with open('../client/public/data/sp_800_171_3_0_0/values.json', 'w') as file:
   json.dump(value_mapping, file, indent=4)

In [ ]:

framework_v2_data = response.json()
framework_v3_data = response.json()

missing_v2_titles = {
    "03.01.13": "Remote Access Confidentiality",
    "03.01.14": "Remote Access Routing",
    "03.01.15": "Privileged Remote Access",
    "03.01.17": "Wireless Access Protection",
    "03.01.19": "Encrypt CUI on Mobile",
    "03.01.21": "Portable Storage Use",
    "03.02.03": "Insider Threat Awareness",
    "03.03.09": "Audit Management",
    "03.04.07": "Nonessential Functionality",
    "03.04.09": "Application Execution Policy",
    "03.05.06": "Identifier Handling",
    "03.05.08": "Password Reuse",
    "03.05.09": "Temporary Passwords",
    "03.05.10": "Cryptographically-Protected Passwords",
    "03.07.01": "Perform Maintenance",
    "03.07.02": "System Maintenance Controls",
    "03.07.03": "Equipment Sanitization",
    "03.08.06": "Cryptographically Protect Portable Storage",
    "03.08.08": "Portable Storage Encryption",
    "03.10.03": "Escort Visitors",
    "03.10.04": "Physical Access Logs",
    "03.10.05": "Manage Physical Access",
    "03.11.03": "Vulnerability Remediation",
    "03.12.04": "System Security Plan",
    "03.13.02": "Security Engineering",
    "03.13.03": "Role Separation",
    "03.13.05": "Public-Access System Separation",
    "03.13.07": "Split Tunneling",
    "03.13.14": "Voice over Internet Protocols",
    "03.13.16": "Data at Rest",
    "03.14.04": "Malicious Code Protection",
    "03.14.05": "System & File Scanning",
    "03.14.07": "Identify Unauthorized Use",
}


for v2_element in v2["response"]["elements"]["elements"]:
   identifier = v2_element["element_identifier"][-8:]
   element_type = v2_element["element_type"]

   if identifier in withdrawn:
      if element_type == 'discussion':
         framework_v2_data["response"]["elements"]["elements"].append(v2_element)
      elif element_type == 'requirement':
         for element in framework_v2_data["response"]["elements"]["elements"]:
            if element["element_identifier"][-8:] == identifier:

               # Add the R2 items to R3
               element["text"] = v2_element["text"]
               element["title"] = v2_element["title"] or (identifier in missing_v2_titles and missing_v2_titles[identifier]) or ""
               framework_v2_data["response"]["elements"]["elements"].append({
                  **v2_element,
                  "element_identifier": f"SR-{identifier}",
                  "element_type": "security_requirement"
               })
               break

In [234]:
v2_only_identifiers = {
    identifier: True for [identifier, item] in value_mapping.items() if 2 in item['revision']
}

v2_only_families = {
    identifier[:5]: True for identifier in v2_only_identifiers.keys()
}

v2_elements = [] 

for element in framework_v2_data["response"]["elements"]["elements"]:
    if element["element_type"] == "withdraw_reason":
        continue

    search = pattern.search(element['element_identifier'])
    if search and search.group() in v2_only_identifiers:
        v2_elements.append(element)
    elif element["element_identifier"] in v2_only_families:
        v2_elements.append(element)

framework_v2_data["response"]["elements"]["elements"] = v2_elements;



In [235]:
# Ensure the directory exists
os.makedirs(os.path.dirname(framework_v2_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_v2_file_path, 'w') as file:
    json.dump(framework_v2_data, file, indent=4)

print(f"JSON data has been stored in {framework_v2_file_path}")

JSON data has been stored in ../client/public/data/sp_800_171_2_0_0/framework.json


In [236]:
%%bash
#! /usr/bin/env bash

sed -i 's|SP_800_171_3_0_0|SP_800_171_2_0_0|g' ../client/public/data/sp_800_171_2_0_0/framework.json

In [237]:
# Ensure the directory exists
os.makedirs(os.path.dirname(framework_v3_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_v3_file_path, 'w') as file:
    json.dump(framework_v3_data, file, indent=4)

print(f"JSON data has been stored in {framework_v3_file_path}")

JSON data has been stored in ../client/public/data/sp_800_171_3_0_0/framework.json
